In [8]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [16]:
def convert_to_prompt(df):
    data_text = df.astype(str)

    prompt_text = f"""
    # 問い合わせデータ
    {data_text}

    上記の問い合わせデータのタイトル、ラベル、説明文から問い合わせの傾向を分析し、Markdown形式で出力してください。
    改善提案は優先度が高い順に3つまで出力してください。

    # 出力形式
    ## 1. 全体サマリー
    ## 2. 改善提案（箇条書き）
    """

    return prompt_text

In [17]:
def get_openai_response(client, prompt_text):
    # 役割とプロンプトを作成
    role = """
    あなたは業務プロセスの改善に精通したビジネスアナリストです。
    企業の社内業務改善をサポートするために、効果的なインサイトを提供します。
    """

    # OpenAI APIの呼び出し
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": role},
            {"role": "user", "content": prompt_text},
        ],
    )

    # 結果を取得
    return response.choices[0].message.content.strip()

In [18]:
def save_result_to_file(result, file_path="問い合わせ改善の提案.md"):
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(result)

In [29]:
# 問い合わせ改善提案ツール
print("処理を開始します。")

# 1. Excelファイルを読み込む
print("データの読み込み")
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='問い合わせ')

# 2. データをプロンプトに変換
print("プロンプト生成")
prompt_text = convert_to_prompt(df)

# 3. OpenAI APIからの応答を取得
print("OpenAI APIからの応答を取得")
result = get_openai_response(client, prompt_text)

# 4. 結果をファイルに保存する関数
print("結果をファイルに保存")
save_result_to_file(result)

print("分析が完了しました。")


処理を開始します。
データの読み込み
プロンプト生成
OpenAI APIからの応答を取得
結果をファイルに保存
分析が完了しました。
